In [1]:
import numpy as np
import open3d as o3d
import os
import sys

# monkey patches visualization and provides helpers to load geometries
sys.path.append('..')
import open3d_tutorial as o3dtut
# change to True if you want to interact with the visualization windows
o3dtut.interactive = not "CI" in os.environ

# Half Edge Mesh

This tutorial outlines the following:
1. How to use `visualization.RenderOption` to render the backface of a mesh.
2. How to use `geometry.AxisAlignedBoundingBox` to crop a mesh. 
3. How to use `utility.Vector3dVector` colorize boundary vertices of a mesh to red. 

In [2]:
def draw_geometries_with_back_face(geometries):
    visualizer = o3d.visualization.Visualizer()
    visualizer.create_window()
    render_option = visualizer.get_render_option()
    render_option.mesh_show_back_face = True
    for geometry in geometries:
        visualizer.add_geometry(geometry)
    visualizer.run()
    visualizer.destroy_window()

## Render the backface of a Mesh

The `draw_geometries_with_back_face` function is responsible for instantiating a `Visualizer` and modifying `RenderOption` to render the backface of a mesh. The `RenderOption` object is retrieved by a call to `get_render_option` and to have the `Visualizer` render the backface of a mesh, `mesh_show_back_face` is set `True`.

In [3]:
# Initialize a HalfEdgeTriangleMesh from TriangleMesh
mesh = o3d.io.read_triangle_mesh("../../test_data/sphere.ply")
bbox = o3d.geometry.AxisAlignedBoundingBox()
bbox.min_bound = [-1, -1, -1]
bbox.max_bound = [1, 0.6, 1]
mesh = mesh.crop(bbox)
het_mesh = o3d.geometry.HalfEdgeTriangleMesh.create_from_triangle_mesh(mesh)
draw_geometries_with_back_face([het_mesh])

## Cropping a Mesh

`geometry.AxisAlignedBoundingBox` is used to create an axis aligned box. The parameters `min_bound` and `max_bound` take an array of cartesian coordinates (x,y,z) and define the size of the bounding box. The bounds are set such that the y-axis of the sphere mesh is cut by a call to `crop`.

In [4]:
# Colorize boundary vertices to red
vertex_colors = 0.75 * np.ones((len(het_mesh.vertices), 3))
for boundary in het_mesh.get_boundaries():
    for vertex_id in boundary:
        vertex_colors[vertex_id] = [1, 0, 0]
het_mesh.vertex_colors = o3d.utility.Vector3dVector(vertex_colors)
draw_geometries_with_back_face([het_mesh])

## Colorize the Boundaries of a Mesh

A call to `geometry.TriangleMesh.get_boundaries` returns a vector of boundaries, where each boundary is a vector of vertices. Each vertex color is represented by an RBG array and `Vector3dVector` is used to convert `vertex_colors` of shape (n, 3) to Open3D format. Finally, the vertex colors are set on the mesh.